In [ ]:
import numpy as np
from bs4 import BeautifulSoup
import pymongo
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import urllib3
import json
import multiprocessing
import time

In [ ]:
client = MongoClient()
db = client['dota']

In [ ]:
pro_id_coll = db.pro_matches_id
pro_matches_full_coll = db.pro_matches_full

In [ ]:
def get_working_proxy():
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    proxy_list_site = 'https://free-proxy-list.net/'
    pm = urllib3.PoolManager(1, 
                             headers=user_agent, 
                             cert_reqs='CERT_REQUIRED')
    prx_req = pm.request('GET', proxy_list_site)
    prx_soup = BeautifulSoup(prx_req.data, 'html.parser')
    proxies_list = []
    for tr in prx_soup.find_all('tr'):
        tmp = []
        for td in tr.find_all('td')[:2]:
            tmp.append(td.text)
        if len(tmp) < 2:
            continue
        proxies_list.append([str(tmp[0]) + ':' + tmp[1]])
    
    proxies_list = proxies_list[:20]
    np.random.seed()
    np.random.shuffle(proxies_list)
    for i, prx in enumerate(proxies_list):
        try:
            print('Getting working proxy...')
            cur_prx_address = 'https://' + str(prx[0]) + '/'
            prx_http = urllib3.ProxyManager(cur_prx_address,
                                            maxsize=1, 
                                            headers=user_agent, 
                                            cert_reqs='CERT_REQUIRED')
            r = prx_http.request('GET', 'https://stackoverflow.com/', timeout=0.7)
            if r.status == 200:
                print('Proxy found.', cur_prx_address)
                return cur_prx_address
            time.sleep(0.1)
        except Exception as err:
            # print(err)
            continue

In [ ]:
def api_get_match(prx_address, match_id):
    url = 'https://api.opendota.com/api/matches/'
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    cur_url = url + str(match_id)
    prx_m = urllib3.ProxyManager(prx_address, headers=user_agent, cert_reqs='CERT_REQUIRED')
    r = prx_m.request('GET', cur_url, timeout=2)
    matches = json.loads(r.data)
    return matches

In [ ]:
def get_new_ids():
    ids_all = [x['match_id'] for x in pro_id_coll.find({}, {'match_id'})]
    ids_stored = [x['match_id'] for x in pro_matches_full_coll.find({}, {'match_id'})]
    need_to_load_ids_bool = np.isin(ids_all, ids_stored, invert=True)
    need_to_load_ids = np.array(ids_all)[need_to_load_ids_bool]
    return need_to_load_ids

In [ ]:
def load_insert(matches_ids, odd_even):
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    prx_address = get_working_proxy()
    pm = urllib3.ProxyManager(proxy_url=prx_address, headers=user_agent, cert_reqs='CERT_REQUIRED')
    
    for i, cur_match_id in enumerate(matches_ids):
        print(i, cur_match_id) if i % 10 == 0 else None
        if cur_match_id % 2 == odd_even:
            try:
                cur_match = api_get_match(prx_address, cur_match_id)
                if 'error' in cur_match.keys():
                    print(cur_match)
                    prx_address = get_working_proxy()
                    continue
                if 'match_id' not in cur_match.keys():
                    print('havent match_id')
                    continue
                pro_matches_full_coll.insert_one(cur_match)
            except Exception as e:
                print(e, cur_match_id)
                continue
        time.sleep(0)

In [ ]:
needed_ids = get_new_ids()
print(len(needed_ids))

pool1 = multiprocessing.Pool(2)
pool2 = multiprocessing.Pool(2)
p1 = pool1.Process(target=load_insert, args=(needed_ids, 0, ))
p2 = pool2.Process(target=load_insert, args=(needed_ids, 1, ))
p1.start()
p2.start()
p1.join()
p2.join()